In [8]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
import plotly.graph_objects as go
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from api import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "pass"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'assets/cat_on_computer.jpg'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date


app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        style={ "display": "flex", "justify-content": "center", "align-items": "center", "gap": "1rem" },
        children=[
            html.H1("Bobby Rust"),
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={ "height": "40px", "width": "40px" })
        ]
    ),
    html.Div(className="buttonRow",
            style = { "display": "flex" },
            children = [
                html.Button(id = "water-button", n_clicks_timestamp = 0, children = "Water Rescue"),
                html.Button(id = "mountain-wilderness-button", n_clicks_timestamp = 1, children = "Mountain & Wilderness Rescue"),
                html.Button(id = "disaster-individual-button", n_clicks_timestamp = 1, children = "Disaster Rescue & Individual Tracking"),
                html.Button(id = "reset-button", n_clicks_timestamp = 2, children = "Reset Filters")]),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                        editable = False,
                        filter_action = "native",
                        sort_action = "native",
                        sort_mode = "multi",
                        column_selectable = False,
                        row_selectable = "single", # Allow single row selection
                        row_deletable = False,
                        selected_columns = [],
                        selected_rows = [0],
                        page_action = "native",
                        page_current = 0,
                        page_size = 10
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
           )
        ]),
        
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('water-button', 'n_clicks_timestamp'),
               Input('mountain-wilderness-button', 'n_clicks_timestamp'),
               Input('disaster-individual-button', 'n_clicks_timestamp'),
               Input('reset-button', 'n_clicks_timestamp')])
def update_dashboard(water_button, mountain_wilderness_button, disaster_individual_button, reset_button):
## FIX ME Add code to filter interactive data table with MongoDB queries
    # Use timestamp values to determine most recently clicked button
    most_recent_click = max(int(water_button), int(mountain_wilderness_button), int(disaster_individual_button), int(reset_button))

    # Take appropriate action based on most recently clicked button
    if most_recent_click == water_button:
        preferred_sex_and_age = {
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {
                "$gt": 26,
                "$lt": 156
            }
        }

        df = pd.DataFrame.from_records(db.read({
            "$or": [
                {
                    "breed": "Labrador Retriever Mix",
                    **preferred_sex_and_age # Unpack operator similar to JavaScript's spread operator [...]
                },
                {
                    "breed": "Chesapeake Bay Retriever",
                    **preferred_sex_and_age 
                },
                {
                    "breed": "Newfoundland",
                    **preferred_sex_and_age                   
                }
            ]
        }))
    elif most_recent_click == mountain_wilderness_button:
        preferred_sex_and_age = {
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gt": 26,
                "$lt": 156
            }
        }

        df = pd.DataFrame.from_records(db.read({
            "$or": [
                {
                    "breed": "German Shepherd",
                    **preferred_sex_and_age
                },
                {
                    "breed": "Alaskan Malamute",
                    **preferred_sex_and_age
                },
                {
                    "breed": "Old English Sheepdog",
                    **preferred_sex_and_age
                },
                {
                    "breed": "Siberian Husky",
                    **preferred_sex_and_age
                },
                {
                    "breed": "Rottweiler",
                    **preferred_sex_and_age
                }
            ]
        }))
    elif most_recent_click == disaster_individual_button:
        preferred_sex_and_outcome = {
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {
                    "$gt": 20,
                    "$lt": 300
                }
        }
        df = pd.DataFrame.from_records(db.read({
            "$or": [
                {
                    "breed": "Doberman Pinscher",
                    **preferred_sex_and_outcome
                },
                {
                    "breed": "German Shepherd",
                    **preferred_sex_and_outcome
                },
                {
                    "breed": "Golden Retriever",
                    **preferred_sex_and_outcome
                },
                {
                    "breed": "Blodhound",
                    **preferred_sex_and_outcome
                },
                {
                    "breed": "Rottweiler",
                    **preferred_sex_and_outcome 
                }
            ]
        }))
    elif most_recent_click == reset_button:
        df = pd.DataFrame.from_records(db.read({}))

    df.drop(columns = ["_id"], inplace = True)

    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')

    return (data)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return

    df_view = pd.DataFrame(viewData)

    is_filtered = len(df) > len(df_view)
    
    if is_filtered:
        return [
            dcc.Graph(figure=go.Figure(data=[go.Pie(labels=["Selected Animals", "Total Animals"], values=[len(df_view), len(df)])]))
        ]
    else:
        breed_counts = df_view['breed'].value_counts()

        total_animals = len(df)

        percentages = (breed_counts / total_animals) * 100

        grouped_breeds = percentages[percentages >= 1]
        other_percentage = percentages[percentages < 1].sum()

        if other_percentage > 0:
            grouped_breeds['Other'] = other_percentage

        return [
            dcc.Graph(
                figure=go.Figure(
                    data=[go.Pie(labels=grouped_breeds.index, values=grouped_breeds)],
                 )
            )
        ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]
app.run_server(debug=True)

Dash app running on http://127.0.0.1:10070/
